install spotipy and tidalapi via pip

In [ ]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

import tidalapi

from math import ceil

create an spotify for developers account

go to the dashboard and create an app. go to edit and add a redirect uri. I use http://localhost:8889/callback
Make sure that no other process is using this localhost port. In my case 8888 was already used by jupyter

We get a public and a private key called client ID and client secret here.

We store them in separate txt files as well as our spotify and tidal user credentials, we could alternatively export them as environmental variables.

In [ ]:
import os

notebook_path = os.path.abspath("")

with open(os.path.join(notebook_path, "clientid.txt")) as fp:
          client_id = fp.read()
        
with open(os.path.join(notebook_path, "clientsecret.txt")) as fp:
          client_secret = fp.read()
        
with open(os.path.join(notebook_path, "spotify_username.txt")) as fp:
          spotify_username = fp.read()
        
with open(os.path.join(notebook_path, "spotify_password.txt")) as fp:
          spotify_password = fp.read()

create a permanent token for accessing the user's library with our app. There are different scopes. We use the "user-library-read" to get the favorite songs and later on change the scope to get the playlists 

In [ ]:
scope = 'user-library-read'
token = util.prompt_for_user_token(spotify_username,
                           scope,
                           client_id=client_id,
                           client_secret=client_secret,
                          redirect_uri='http://localhost:8889/callback')

In [ ]:
sp = spotipy.Spotify(auth=token)


We give a maximum number of playlists and a maximum number of favorite songs that we are going to import. There is probably a more beautiful way to access this number via API ;)

In [ ]:
MAX_PLAYLISTS = 200
MAX_USER_SAVED_TRACKS = 50000

Get the favorite songs from spotify

In [ ]:
spotify_my_songs = []

for chunk in range(ceil(MAX_USER_SAVED_TRACKS/50)):
    
    user_saved_tracks_resp = sp.current_user_saved_tracks(limit = 50, offset = 50*chunk)

    for item in user_saved_tracks_resp['items']:
        spotify_my_songs.append((item["track"]["name"], item["track"]["artists"][0]["name"]))
    



In [ ]:
print("Number of songs found in Spotify Favorites: ", len(spotify_my_songs))

Search for each one on tidal and get its id, post the ones that are not found on tidal. We simply take the first search result as the right song.

In [ ]:
tidal_track_ids = []

for my_song in spotify_my_songs:
    tidal_search_results = session.search(my_song[0] + " " + my_song[1])
    
    if len(tidal_search_results["tracks"]) == 0:
        print(my_song[0], " - ", my_song[1], " not found on tidal")
            
    for found_track in tidal_search_results["tracks"]:
        tidal_track_ids.append(found_track.id)
        break

We need to start a tidal API Session, follow the instructions below

In [ ]:
session = tidalapi.Session()

session.login_oauth_simple()
print(session.check_login())

We further need the User ID to get the favorites

In [ ]:
tidal_user_id = session.user.id
favs = tidalapi.user.Favorites(session, tidal_user_id)

In [ ]:
successes = 0
for el in tidal_track_ids:
    try:
        successes += favs.add_track(el)
    except HTTPError:
        continue

In [ ]:
print(successes, "Tracks added to the favorites")

Now import the Playlists! We create a new token for the playlist access

In [ ]:
playlist_scope = 'playlist-read-private'
token = util.prompt_for_user_token(spotify_username,
                           playlist_scope,
                           client_id=client_id,
                           client_secret=client_secret,
                          redirect_uri='http://localhost:8889/callback')
sp = spotipy.Spotify(auth=token)

Get the content of my playlists in spotify

In [ ]:

spotify_playlists_dicts = []
for chunk in range(ceil(MAX_PLAYLISTS/50)):
    sp_pl_d = sp.current_user_playlists(limit = 50, offset = 50*chunk)
    spotify_playlists_dicts.append(sp_pl_d)
    
spotify_playlists_items = [el for subl in spotify_playlists_dicts for el in subl["items"]]

In [ ]:
print(f"There are {len(spotify_playlists_items)} playlists to transfer")

The following code creates for each spotify playlist a tidal playlists and adds songs. We find the songs on tidal by simply searching for the artist and the track name provided by spotify and take the first search result as above for the favorite songs

In [ ]:
for spotify_playlist in spotify_playlists_items:
    
    
    
    # create tidal_playlist
    tidal_playlist = session.user.create_playlist(spotify_playlist["name"], f"{spotify_playlist['name']} imported from spotify")
    
    # get the tracks in the playlist 
    tracks = sp.user_playlist(spotify_username, spotify_playlist["id"], fields="tracks,next")
    
    tidal_track_ids = []
    
    # iterate over all tracks in the spotify playlist, search for them on tidal and get their IDs
    for track in tracks["tracks"]["items"]:
        spotify_artist_name = track["track"]["artists"][0]["name"]
        spotify_track_name = track["track"]["name"]
        
        # search on tidal
        tidal_search_results = session.search(spotify_artist_name + " " + spotify_track_name)
        
        # just add the first search result!
        
        if len(tidal_search_results["tracks"]) == 0:
            print(spotify_artist_name, " - ", spotify_track_name, " not found on tidal")
        for found_track in tidal_search_results["tracks"]:
            tidal_track_ids.append(found_track.id)
            break
            
        # There could also be a more sophisticated approach by matching artist names and track names etc.
#             if found_track.artist.name == spotify_artist_name and found_track.name == spotify_track_name:
#                 tidal_track_ids.append(found_track.id)
#                 break
#         else:
#             print("no match for ", spotify_artist_name, spotify_track_name)
    if len(tidal_track_ids) == 0:
        print("!!!", spotify_playlist["name"], " - no tracks found")
        continue
    tidal_playlist.add(tidal_track_ids)
    print("   ", spotify_playlist["name"], " - playlist created")